In [1]:
import numpy as np
import pandas as pd

from models import *
from model_training import *
from utils import *

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# variable init
batch_size = 32
val_split = 0.3
shuffle_dataset = True
random_seed = 123
visualization = True
num_epochs = 5
valid_loss_min = np.Inf
model_name = 'resnet'
num_classes = 2
input_dim = 8

In [3]:
# random image dataset for 8 dimension image
class random_dataset(Dataset):
    def __init__(self):
        self.image = np.random.rand(5000,224,224,8)
        self.labels = np.random.choice([0, 1], size=(5000,), p=[0.6,0.4])

    def __getitem__(self, index):
        image_1 = self.image[index]
        labels_1 = self.labels[index]
        image_2 = np.transpose(image_1, (2,1,0))
        return image_2, labels_1
    
    def __len__(self):
        return len(self.labels)

In [4]:
# Creating training and validation dataset
dataset = random_dataset()
data_size = len(dataset)

indices = list(range(data_size))
split = int(np.floor(val_split * data_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Create train ad val data loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset,
                                            batch_size=batch_size,
                                            sampler=train_sampler,
                                            num_workers=8,
                                            pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset,
                                            batch_size=batch_size,
                                            sampler=valid_sampler,
                                            num_workers=8,
                                            pin_memory=True)

In [5]:
print('11')
# resnet, vgg, densenet, alexnet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft, input_size = initialize_model(model_name, num_classes, 8, use_pretrained=True)

# Put the model on the device:
model_ft = model_ft.to(device)

11


/home/rakesh.kumar/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
# define loss function
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_ft.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)

In [7]:
## model training
## model_train function save model automatically as model_multi_dim.pth
model_train(model_ft, optimizer, criterion, lr_scheduler, device, train_loader, val_loader, n_epochs=num_epochs).run()

Epoch 0: | Training_Loss: 0.68115                     | Training_Acc: 57.799
Epoch 0: | Validation_Loss: 0.67693 | Validation_Acc: 59.872
Validation loss decreased (inf --> 0.676928).  Saving model ...
Epoch 1: | Training_Loss: 0.67095                     | Training_Acc: 59.426
Epoch 1: | Validation_Loss: 0.67996 | Validation_Acc: 59.717
EarlyStopping counter: 1 out of 15
Epoch 2: | Training_Loss: 0.65613                     | Training_Acc: 60.033
Epoch 2: | Validation_Loss: 0.68347 | Validation_Acc: 57.413
EarlyStopping counter: 2 out of 15
Epoch 3: | Training_Loss: 0.61712                     | Training_Acc: 65.101
Epoch 3: | Validation_Loss: 0.73543 | Validation_Acc: 59.253
EarlyStopping counter: 3 out of 15
Epoch 4: | Training_Loss: 0.50431                     | Training_Acc: 75.957
Epoch 4: | Validation_Loss: 0.89973 | Validation_Acc: 43.789
EarlyStopping counter: 4 out of 15
Final best acc_score = 5987.2002%


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track